## Tic Tac Toe AI ##


In the tic-tac-toe code below, I created unbeatable AI.

I allowed the player to choose the skill level.

    1. Random Player - Skill Level Easy
    2. Utility Based Agent and Goal Based Agent (PA1) - Skill Level Medium
    3. Skill Level Hard 1 - Implemented the MiniMax Algorithm
    4.  Skill Level Hard 2 - Implemented the MiniMax Algorithm

In [10]:
class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]  # 3x3 Tic-Tac-Toe board
        self.current_player = 'X'  # 'X' or 'O'
        self.win_combinations = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columns
            [0, 4, 8], [2, 4, 6]              # Diagonals
        ]

    def display_board(self):
        for i in range(3):
            print('|'.join(self.board[i*3:(i+1)*3]))
            if i < 2:
                print('-----')

    def is_winner(self, player):
        win_states = [
            [self.board[0], self.board[1], self.board[2]],
            [self.board[3], self.board[4], self.board[5]],
            [self.board[6], self.board[7], self.board[8]],
            [self.board[0], self.board[3], self.board[6]],
            [self.board[1], self.board[4], self.board[7]],
            [self.board[2], self.board[5], self.board[8]],
            [self.board[0], self.board[4], self.board[8]],
            [self.board[2], self.board[4], self.board[6]],
        ]
        return [player, player, player] in win_states

    def is_board_full(self):
        return ' ' not in self.board

    def is_game_over(self):
        return self.is_winner('X') or self.is_winner('O') or self.is_board_full()

    def make_move(self, position, player):
        if self.board[position] == ' ':
            self.board[position] = player
            return True
        return False

    def get_available_positions(self):
        return [i for i, x in enumerate(self.board) if x == ' ']

    def switch_player(self):
        self.current_player = 'O' if self.current_player == 'X' else 'X'

    def utility(self, player):
        if self.is_winner(player):
            return 1
        elif self.is_winner('O' if player == 'X' else 'X'):
            return -1
        return 0

    def to_move(self):
        return self.current_player


In [2]:
class Player:
    def __init__(self, symbol):
        self.symbol = symbol

    def get_move(self, game):
        pass

class HumanPlayer(Player):
    def get_move(self, game):
        valid_move = False
        while not valid_move:
            try:
                move = int(input(f"Player {self.symbol}, enter your move (0-8): "))
                if move < 0 or move > 8:
                    raise ValueError
                valid_move = game.make_move(move, self.symbol)
            except ValueError:
                print("Invalid move. Please try again.")
        return move


In [3]:
import random

class RandomPlayer(Player):
    def __init__(self, symbol):
        super().__init__(symbol)

    def get_move(self, game):
        available_positions = game.get_available_positions()
        if available_positions:
            move = random.choice(available_positions)
            game.make_move(move, self.symbol)
            return move
        return None


In [4]:
class UtilityBasedPlayer(Player):
    def __init__(self, symbol):
        super().__init__(symbol)

    def get_move(self, game):
        best_score = float('-inf')
        best_move = None

        for move in game.get_available_positions():
            game.make_move(move, self.symbol)
            score = self.evaluate_board(game)
            game.board[move] = ' '  # Undo move

            if score > best_score:
                best_score = score
                best_move = move

        game.make_move(best_move, self.symbol)
        return best_move

    def evaluate_board(self, game):
        # simple heuristic 
        score = 0
        for combo in game.win_combinations:
            values = [game.board[i] for i in combo]
            if values.count(self.symbol) == 2 and values.count(' ') == 1:
                score += 1
        return score



In [5]:
class GoalBasedPlayer(Player):
    def __init__(self, symbol):
        super().__init__(symbol)

    def get_move(self, game):
        # 1) Win if possible, 2) Block opponent if they can win next move, 3) Take center, 4) Take corners
        for move in game.get_available_positions():
            game.make_move(move, self.symbol)
            if game.is_winner(self.symbol):  # Goal 1: Win
                return move
            game.board[move] = ' '  # Undo move

        opponent = 'O' if self.symbol == 'X' else 'X'
        for move in game.get_available_positions():
            game.make_move(move, opponent)
            if game.is_winner(opponent):  # Goal 2: Block opponent
                game.board[move] = ' '  # Undo move
                game.make_move(move, self.symbol)
                return move
            game.board[move] = ' '  # Undo move

        # Goal 3: Take center
        if game.board[4] == ' ':
            game.make_move(4, self.symbol)
            return 4

        # Goal 4: Take corners
        for corner in [0, 2, 6, 8]:
            if game.board[corner] == ' ':
                game.make_move(corner, self.symbol)
                return corner

        # Fallback: Choose a random move
        return random.choice(game.get_available_positions())

In [6]:
class MiniMaxPlayer(Player):
    def __init__(self, symbol):
        super().__init__(symbol)

    def get_move(self, game):
        _, move = self.minimax(game, True)
        game.make_move(move, self.symbol)
        return move

    def minimax(self, game, is_maximizing):
        if game.is_game_over():
            return game.utility(self.symbol), None

        if is_maximizing:
            best_score = float('-inf')
            best_move = None
            for move in game.get_available_positions():
                game.make_move(move, self.symbol)
                score, _ = self.minimax(game, False)
                game.board[move] = ' '  # Undo move
                if score > best_score:
                    best_score = score
                    best_move = move
            return best_score, best_move
        else:
            best_score = float('inf')
            best_move = None
            opponent_symbol = 'O' if self.symbol == 'X' else 'X'
            for move in game.get_available_positions():
                game.make_move(move, opponent_symbol)
                score, _ = self.minimax(game, True)
                game.board[move] = ' '  # Undo move
                if score < best_score:
                    best_score = score
                    best_move = move
            return best_score, best_move


In [7]:
class AlphaBetaPlayer(Player):
    def __init__(self, symbol):
        super().__init__(symbol)

    def get_move(self, game):
        _, move = self.alpha_beta_decision(game)
        game.make_move(move, self.symbol)
        return move

    def alpha_beta_decision(self, game):
        value, move = self.max_value(game, float('-inf'), float('inf'))
        return value, move  # returns a tuple

    def max_value(self, game, alpha, beta):
        if game.is_game_over():
            return game.utility(self.symbol), None

        v = float('-inf')
        best_move = None
        for move in game.get_available_positions():
            game.make_move(move, self.symbol)
            v2, _ = self.min_value(game, alpha, beta)
            game.board[move] = ' '  # Undo move
            if v2 > v:
                v, best_move = v2, move
            if v >= beta:
                return v, best_move
            alpha = max(alpha, v)
        return v, best_move  

    def min_value(self, game, alpha, beta):
        if game.is_game_over():
            return game.utility(self.symbol), None

        v = float('inf')
        best_move = None
        opponent_symbol = 'O' if self.symbol == 'X' else 'X'
        for move in game.get_available_positions():
            game.make_move(move, opponent_symbol)
            v2, _ = self.max_value(game, alpha, beta)
            game.board[move] = ' '  # Undo move
            if v2 < v:
                v, best_move = v2, move
            if v <= alpha:
                return v, best_move 
            beta = min(beta, v)
        return v, best_move 

In [16]:
def main():
    game = TicTacToe()
    player1 = HumanPlayer('X')

    # Prompt the user to choose the difficulty level
    while True:
        choice = input("Choose the difficulty level - Easy (Random), Medium (Utility/Goal-Based), Hard 1 (MiniMax), or Hard 2 (Alpha Beta): ").strip().lower()
        if choice == 'easy' or choice == 'random':
            player2 = RandomPlayer('O')
            break
            
        elif choice == 'medium':
        # You can further ask the user to choose between Utility-Based or Goal-Based
            sub_choice = input("Choose Medium Type - Utility-Based or Goal-Based: ").strip().lower()
            if sub_choice == 'utility':
                player2 = UtilityBasedPlayer('O')
            elif sub_choice == 'goal':
                player2 = GoalBasedPlayer('O')
            else:
                print("Invalid choice. Defaulting to Utility-Based.")
                player2 = UtilityBasedPlayer('O')
            break
            
        elif choice == 'hard 1' or choice == 'minimax':
            player2 = MiniMaxPlayer('O')
            break
            
        elif choice == 'hard 2' or choice == 'alpha beta':
            player2 = AlphaBetaPlayer('O')
            break
            
        else:
            print("Invalid choice. Please choose 'Easy', 'Hard 1', or 'Hard 2'.")
            
       

    while not game.is_game_over():
        game.display_board()
        if game.to_move() == player1.symbol:
            player1.get_move(game)
        else:
            player2.get_move(game)
        game.switch_player()

    game.display_board()
    if game.is_winner(player1.symbol):
        print("Player X wins!")
    elif game.is_winner(player2.symbol):
        print("Player O wins!")
    else:
        print("It's a draw!")

if __name__ == "__main__":
    main()


Choose the difficulty level - Easy (Random), Medium (Utility/Goal-Based), Hard 1 (MiniMax), or Hard 2 (Alpha Beta): Hard 1
 | | 
-----
 | | 
-----
 | | 
Player X, enter your move (0-8): 4
 | | 
-----
 |X| 
-----
 | | 
O| | 
-----
 |X| 
-----
 | | 
Player X, enter your move (0-8): 2
O| |X
-----
 |X| 
-----
 | | 
O| |X
-----
 |X| 
-----
O| | 
Player X, enter your move (0-8): 1
O|X|X
-----
 |X| 
-----
O| | 
O|X|X
-----
O|X| 
-----
O| | 
Player O wins!
